In [1]:
import networkx as nx
import neo4j
from neo4j import GraphDatabase, basic_auth
import json
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
driver = neo4j.GraphDatabase.driver(
    "neo4j://localhost:7687", auth=("neo4j", "flowerbwoy"))

In [3]:
df = pd.read_csv('local_black.csv')
df

,#,Name,Black Owned Confirmed?,Area,Direct Venmo/Donation Option/Giftcard,Cuisine/ Shop type,Website,Instagram,FB,Twitter,YouTube,Delivery Options,Combined Address,Street Address,City,State,Zipcode,Email Address,Phone
0,1.0,Rock City Pizza,Confirmed,Allston,NaN,American,https://rockcitypizzeria.com/,https://www.instagram.com/rockcitypizzaallston/,https://www.facebook.com/RockCityPizzeria/,NaN,NaN,NaN,"568 Cambridge St, Boston, MA 02134",568 Cambridge St,Allston,MA,2134,NaN,NaN
1,4.0,TamBos Kitchen,Unconfirmed,Avon,NaN,West African,https://tamboskitchen.com/,https://www.instagram.com/tamboskitchen/,https://www.facebook.com/tamboskitchen/,NaN,https://www.youtube.com/channel/UCfh-OI9DB3Wtm...,Local,"490 W Main St, Avon, MA 02322",490 W Main St,Avon,MA,2322,NaN,NaN
2,5.0,Restaurante Cesaria,Confirmed,Boston,NaN,African/ Cape Verdean,https://www.restaurantcesaria.com/,https://www.instagram.com/restaurantecesaria/,https://www.facebook.com/RestaurantCesaria/,NaN,NaN,"Doordash, grubhub, ubereats","266 Bowdoin St, Boston, MA 02122",266 Bowdoin St,Boston,MA,2122,NaN,NaN
3,6.0,Bred Gourmet,Confirmed,Boston,http://www.bredgourmet.com/shopbred/bred-gift-...,American,http://www.bredgourmet.com/,https://www.instagram.com/bredgourmet/,https://www.facebook.com/BREDGOURMET/,https://twitter.com/BredGourmet,NaN,Grubhub,"2255 Dorchester Ave, Boston, MA 02124",2255 Dorchester Ave,Boston,MA,2124,NaN,6176980103
4,7.0,Darryl's Corner Bar and Kitchen,Confirmed,Boston,https://www.toasttab.com/darryls-corner-bar-ki...,American,http://dcbkboston.com/,https://www.instagram.com/dcbkboston/,https://www.facebook.com/DCBKBoston/,https://twitter.com/DCBKBoston,NaN,Grubhub,"604 Columbus Ave, Boston, MA 02118",604 Columbus Ave,Boston,MA,2118,NaN,6175361100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,151.0,Brazo Fuerte,Unconfirmed,Watertown,NaN,Brewery,http://brazo-fuerte.squarespace.com/,https://www.instagram.com/brazofuertebeer/,https://www.facebook.com/brazofuertebeer/,https://twitter.com/BrazoFuerteBeer,NaN,local,NaN,NaN,NaN,MA,NaN,NaN,NaN
151,152.0,Viva Mi Arepa,Unconfirmed,West Roxbury,NaN,Haitian/Venezuelan,https://www.vivamiareparestaurant.com/,NaN,NaN,NaN,NaN,"Grubhub, uber eats","5197 Washington Street\nBoston, MA 02132",5197 Washington Street\nBoston,MA 02132,MA,2132,NaN,NaN
152,153.0,Belmont Vegetarian,Unconfirmed,Worcester,NaN,Caribbean,https://www.belmontvegetarian.com/index.html,https://www.instagram.com/belmontvegetarian_/,https://www.facebook.com/BelmontVegetarianRest...,https://twitter.com/belmontveg1,NaN,Pick up,"157 Belmont St, Worcester, MA 01605",157 Belmont St,Worcester,MA,1605,NaN,NaN
153,154.0,Fatima's Cafe,Unconfirmed,Worcester,NaN,East African / Ethiopian,http://www.fatimascafe.com/,NaN,NaN,NaN,NaN,"Pick up, catering","43 West Boylston St, Worcester, MA",43 West Boylston St,Worcester,MA,NaN,NaN,NaN


In [68]:
def build_graph():
    driver.close()
    with driver.session() as session:
        cypher1 = """
            LOAD CSV WITH HEADERS FROM 'file:///local_black.csv' AS Line
            UNWIND split(Line.`Cuisine/ Shop type`, ' / ') AS type
            MERGE (s:Type {name: type})
            """
        cypher2 = """
            LOAD CSV WITH HEADERS FROM 'file:///local_black.csv' AS Line
            CREATE(:Business {id: toInteger(Line.`#`),
                              name: Line.Name, 
                              confirmed: Line.`Black Owned Confirmed?`,
                              area: Line.Area,
                              direct_aid: Line.`Direct Venmo/Donation Option/Giftcard`,
                              cuisine_or_type: Line.`Cuisine/ Shop type`,
                              url: Line.Website,
                              instagram: Line.Instagram,
                              fb: Line.FB,
                              twitter: Line.Twitter,
                              delivery: Line.`Delivery Options`,
                              address: Line.`Combined Address`,
                              phone: Line.Phone})      
        """
        cypher3 = """
            MATCH(n) WHERE(n.confirmed = 'Unconfirmed') DELETE n
        """
        session.run(cypher1)
        session.run(cypher2)

    driver.close()

def map_cuisine():
    
    cuisines = []
    driver.close()
    with driver.session() as session2:
        result = session2.run('MATCH(a:Type) RETURN a.name as name')
        for line in result:
            cuisines.append(line['name'])
    
    
    #c_set = set([n for n in df['Cuisine/ Shop type']])
    print(cuisines)
    c_set = set([n for n in cuisines])
    for c in c_set:
        for index, row in df.iterrows():
            if c in str(row['Cuisine/ Shop type']):
                #driver.session().run("MERGE(:Type {name: $node2})", node2=c)
                #driver.close()
                driver.session().run("""
                    MATCH(a:Business),(b:Type) 
                    WHERE a.name = $node1 AND b.name = $node2
                    MERGE (b)-[r:RELTYPE]->(a)
                """, node1=row['Name'], node2=c)
                driver.close()
                
    driver.close()
    
def create_node(session, driver, row):
    cypher = """
        MERGE (n:Business {name: $row.Name}, {Black}),
    
    """
    session.run("MERGE (n:Business {name: $node})", node=node)

def create_relationship(session, tx, node1, node2):
    session.run("""
        MATCH (a:Topic),(b:Topic) 
        WHERE a.name = $node1 AND b.name = $node2 
        MERGE (a)-[r:IS_RELATED_TO]->(b)""", node1=node1, node2=node2)

def clear_database():
    query = """
        MATCH (n)
        DETACH DELETE n
        """
    with driver.session() as session:
        session.run(query)

In [69]:
driver.close()
clear_database()
build_graph()
map_cuisine()
#map_cuisine()

['Haitian', 'American', 'African', 'Cape Verdean', 'Bakery', 'Bakery ', 'Caribbean', 'Italian', 'Sandwiches', 'Senegalese', 'Trinidadian', 'Wine Shop', 'Ethiopian', 'Barbecue', 'Breakfast', 'Angolan', 'Haitian ', 'Cape Verde', 'Coffee', 'Southern/Asian Fusion', 'Somali', 'Mediterranean', 'Jamaican', 'American/Hot dogs', 'Breakfast & Lunch - Local Ingredients', 'Soul food', 'Cape Verdean ', 'East African', 'West African', 'Bookstore', 'Filipinix American', 'Bakery/Coffee Shop', 'BBQ', 'Latin/Cuban', 'Beverage Company', 'Diner', 'Ethiopian/Cafe', 'Juice/Beverages', 'Latin', 'Dominican', 'Tacos', 'Puerto Rican', 'Creole', 'Tunisian', 'Moroccan', 'North African', 'Pizzeria', 'Mexican', 'Seafood', 'Clothing Company', 'Brewery ', 'Massage', 'Catering Company', 'Burgers', 'Nigerian', 'African and Caribbean', 'Juice Bar', 'Latin American', 'Carribean - Haitian ', 'Egyptian/North African', 'Ice Cream', 'Brazilian', 'Venezuelan ', 'Ghanaian', 'West African ', 'Vegan/Vegetarian']
